In [1]:
import pandas as pd
import os


path = "./dll_analysis/malware"
filename = "fixed_rules.csv"
file_to_scan = os.path.join(path,filename)
df1 = pd.read_csv(file_to_scan)

In [2]:
df1.head()


,Unnamed: 0,Exe name,Family,first_seen,last_seen,total,positives,ssdeep,imphash,yara_detected,yara_matching_rules
0,0,05c0f641bc943f8c1b6c1eddafebc2ee107676b3b2c57a...,locky,2016-11-02 08:42:23,2016-11-02 08:42:23,57.0,32.0,3072:6M0OkZhEYDToI3aS4OHjNobuQih7Pu/X8iJdpxsnL...,cba475024652ed6788f8d77ae1a30155,True,['../yara/rules-master/malware/h3x2b/yara-rule...
1,1,1524b86c0e00ac5b93af3db19265a79ec269ef67573802...,locky,2016-10-31 12:30:02,2016-11-01 19:45:57,57.0,37.0,3072:EH5NBtlf2OkjN9rbNIGD9sIogzNGHPXk8wr8rFnjG...,a23d5ed45b5bb2d45988b4d790bb2fb7,True,['../yara/rules-master/malware/x64dbg/yarasigs...
2,2,d82b557fc694192fd928fbf916a979c0838eec6d42f334...,razy,2016-11-04 16:48:17,2016-11-04 16:48:17,57.0,20.0,3072:Q9FusUCqe04D+Dmo5NdPQ1IYkze2b1OBGcXH3nXPi...,3736395e43e13030dde5ba3bc3da56d5,False,[]
3,3,33fbae9c05b6e46aa3360bd20d2f39216b91b6c167595a...,locky,2016-11-04 14:36:51,2016-11-04 14:36:51,56.0,39.0,3072:mLhD1etmEcttEm7zj/FDVaRj0KmClAnXH3nXPqn:c...,3736395e43e13030dde5ba3bc3da56d5,False,[]
4,4,58114b0f4eb890f498b96eed084c2f341f711dbf9fc941...,locky,2016-11-02 20:17:59,2016-12-06 20:27:17,57.0,44.0,3072:nnpfH5egxO5TIB9FAy1ZsMKNDO5kwMAFbjSlWmVrT...,9cd36be5889827aef4f69402aa78a2ba,False,[]


In [3]:
import pefile
import ssdeep

m1 = df1.loc[:,["Exe name", "Family", "first_seen", "imphash"]]
m1["first_seen:month"] = pd.to_datetime(m1["first_seen"]).dt.to_period("M")

In [4]:
del m1["first_seen"]

In [5]:
m1.head()

,Exe name,Family,imphash,first_seen:month
0,05c0f641bc943f8c1b6c1eddafebc2ee107676b3b2c57a...,locky,cba475024652ed6788f8d77ae1a30155,2016-11
1,1524b86c0e00ac5b93af3db19265a79ec269ef67573802...,locky,a23d5ed45b5bb2d45988b4d790bb2fb7,2016-10
2,d82b557fc694192fd928fbf916a979c0838eec6d42f334...,razy,3736395e43e13030dde5ba3bc3da56d5,2016-11
3,33fbae9c05b6e46aa3360bd20d2f39216b91b6c167595a...,locky,3736395e43e13030dde5ba3bc3da56d5,2016-11
4,58114b0f4eb890f498b96eed084c2f341f711dbf9fc941...,locky,9cd36be5889827aef4f69402aa78a2ba,2016-11


In [6]:
UniqueFamilies = m1.Family.unique()

In [7]:
df_dict={el: pd.DataFrame() for el in UniqueFamilies}

In [8]:
for family in UniqueFamilies:
    mf = m1[:][m1.Family == family]
    """ Performing cartesian product of the dataset with itself
    ( it will be useful to use the apply function over the same row of the dataframe).
    Each row will now contain two samples of the same family 
    (too complex and, probably, useless to perform it over all the samples, independently on the family)"""
    cp = mf.merge(mf, how='outer', on = "Family")
    cp["IAT equality"] = cp.apply(lambda x: int(x["imphash_x"]==x["imphash_y"]), axis=1)
    df_dict[family] = cp
    del mf


In [10]:
import gc
gc.collect()
print(path)

./dll_analysis/malware


In [11]:
import matplotlib.pyplot as plt


for family in UniqueFamilies:
    if(df_dict[family].shape[0]<2):
        continue
    df_dict[family] = df_dict[family].pivot(index= "Exe name_x", columns="Exe name_y", values="IAT equality")
    ax = plt.pcolor(df_dict[family])
    ax.get_figure().savefig(path+"/heatmap/iat_heatmap/"+family+"_heatmap", dpi=900)
    plt.clf()

In [12]:
for family in UniqueFamilies:
    df_dict[family].to_csv(path+"/heatmap/iat_heatmap/"+family+"_heatmap.csv")

In [28]:
df_dict['teslacrypt'].values 

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [30]:
for el in range(df_dict["teslacrypt"].shape[0]):
    print(df_dict["teslacrypt"].values[el][el])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
